<a href="https://colab.research.google.com/github/abhisaurabh0308/Building-an-LLM/blob/main/llm_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install requests gTTS


In [12]:
import requests
from gtts import gTTS
from IPython.display import Audio
import os


In [13]:
LLM_API_KEY = "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"  # Replace with your Groq API key
WEATHER_API_KEY = "ca40e674dfb12f4c2594108ae26161f4"  # Replace with your OpenWeatherMap key
LLM_API_URL = "https://api.groq.com/openai/v1/chat/completions"


In [14]:
def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather_desc = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The current weather in {city} is {weather_desc} with a temperature of {temp}°C."
    else:
        return f"Couldn't fetch the weather for {city}. Please check the city name."


In [15]:
def get_llm_reply(instruction, user_input):
    headers = {
        "Authorization": f"Bearer {LLM_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "meta-llama/llama-4-scout-17b-16e-instruct",
        "messages": [
            {"role": "system", "content": instruction},
            {"role": "user", "content": user_input}
        ]
    }
    response = requests.post(LLM_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error {response.status_code}: {response.text}"


In [16]:
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output.mp3")
    return Audio("output.mp3", autoplay=True)


In [17]:
def chat(instruction, user_input):
    if "weather" in user_input.lower():
        # Try to extract city name
        try:
            words = user_input.lower().split()
            if "in" in words:
                city_index = words.index("in") + 1
                city = " ".join(words[city_index:]).title()
            else:
                raise ValueError("No city mentioned.")
            reply = get_weather(city)
        except:
            reply = "Please specify a city after the word 'in'. For example: 'What is the weather in Delhi?'"
    else:
        reply = get_llm_reply(instruction, user_input)

    print("🤖:", reply)
    return speak(reply)


In [20]:
instruction = "You are a helpful assistant."
user_input = input("💬 Ask something: ")
chat(instruction, user_input)


💬 Ask something: Tell me the weather in New Delhi
🤖: The current weather in New Delhi is haze with a temperature of 35.09°C.
